In [ ]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
import vizro.plotly.express as px
from data_cleaning_and_utilities import plot_preset, get_earth_colorscale

# Data loading

In [12]:
path = ("../data/cleaned_main.csv")
df_main = pd.read_csv(path)
path = "../data/cleaned_variants.csv"
df_variants = pd.read_csv(path)

### Preview of df_main

In [13]:
df_main.head(2)

,main_category,subcategory,master_name,review_amount,rating,size,details,True to size (size_ag),Too Big (size_ag),Small (size_ag),Too Small (size_ag),Not specified (size_ag),Big (size_ag),Not specified (experience),Quality (experience),Stylish (experience),Comfortable (experience),Verified Buyer,Verified Reviewer,Unverified
0,Woman's,Womens-sandals,Lumina,8,4.9,Big,.5 inch heel height; Suede; Textile lining; Sy...,3.0,1.0,0.0,0.0,1.0,3.0,4.0,3.0,3.0,3.0,7.0,0.0,1.0
1,Woman's,Womens-sandals,Linear,18,4.9,True to size,1 inch heel height; Suede upper material with ...,10.0,4.0,1.0,0.0,2.0,1.0,13.0,3.0,5.0,4.0,11.0,4.0,3.0


### 🔍 Schema: `df_main`

| Column                        | Description                                                  |
|-------------------------------|--------------------------------------------------------------|
| `main_category`               | High-level product category                                  |
| `subcategory`                 | Specific product subcategory                                 |
| `master_name`                 | Base product name                                            |
| `review_amount`               | Number of reviews for the product                            |
| `rating`                      | Average customer rating (1–5 scale)                          |
| `size`                        | Average size fit (site-generated)                            |
| `details`                     | Descriptive product details                                  |

**Size Feedback Columns (`(size_ag)`) - these columns are counts of user-selected size-related tags from individual scraped reviews, such as:**
- `True to size (size_ag)`, `Too Big (size_ag)`, `Small (size_ag)`, `Too Small (size_ag)`, `Not specified (size_ag)`, `Big (size_ag)`

**Experience Feedback Columns (`(experience)`) - These columns are counts of user-selected experience-related tags from scraped reviews, such as:**
- `Not specified (experience)`, `Quality (experience)`, `Stylish (experience)`, `Comfortable (experience)`

**Verification Columns - indicate customer verification status:**
- `Verified Buyer`, `Verified Reviewer`, `Unverified`

### Preview of df_variants

In [14]:
df_variants.head(2)

,main_category,subcategory,master_name,full_product_name,product_link,price,color
0,Woman's,Womens-sandals,Lumina,Lumina Brown suede,https://www.stevemadden.com/collections/womens...,79.95,Brown suede
1,Woman's,Womens-sandals,Linear,Linear Taupe suede,https://www.stevemadden.com/collections/womens...,109.95,Taupe suede


### 🔍 Schema: `df_variants`

| Column               | Description                                   |
|----------------------|-----------------------------------------------|
| `main_category`      | High-level product category                   |
| `subcategory`        | Specific product subcategory                  |
| `master_name`        | Base product name                             |
| `full_product_name`  | Full variant name (with color)                |
| `product_link`       | Link to product page                          |
| `price`              | Product price (USD)                           |
| `color`              | Product color                                 |

In [15]:
# Set up plot preset
pio.templates.default = "vizro_dark"
earth_palette = get_earth_colorscale()


# Exploratory Data Analysis


### Unique Products per Subcategory



In [16]:
# Count unique products by subcategory
df = df_main.groupby("subcategory")["master_name"].count().reset_index()
df = df.sort_values(by= "master_name", ascending= False)

# Create horizontal bar chart 
fig = px.bar(df, y = "subcategory", x = "master_name", color= "master_name", color_continuous_scale="Earth", title = "Number of Unique Products by Subcategory")
# Apply layout and styling
fig = plot_preset(fig)
fig.update_layout(
    xaxis_title="Number of products",
    yaxis_title="Subcategory",
    height = 700
);


### Amount of subcategories


In [17]:
print(f"Number of unique subcategories - {df_main["subcategory"].nunique()}")

Number of unique subcategories - 36



### Distribution of Reviews by Rating


In [18]:
# Aggregate review counts per rating to analyze distribution of customer feedback
df = df_main.groupby("rating")["review_amount"].sum().reset_index()
df = df.sort_values(by= "rating", ascending= True)

# Create bar chart showing total review count per rating
fig = px.bar(df, y = "review_amount", x = "rating", color= "review_amount", color_continuous_scale="Earth", title = "Distribution of Reviews by Rating")
# Update axis labels
fig = plot_preset(fig)
fig.update_layout(
    xaxis_title="Rating",
    yaxis_title="Number of Reviews",
);


### Top 10 Most Reviewed Products


In [19]:
# Add name_and_subcategory column 
df_main["name_and_subcategory"] = df_main["master_name"] + " " + df_main["subcategory"]
# Select top 10 products by total review count, ensuring unique product names
df_top10 = df_main.sort_values("review_amount", ascending=False).drop_duplicates(subset=["master_name"]).iloc[:10]

# Initialize a Plotly figure for the lollipop chart visualization
fig = go.Figure()

# Add scatter trace combining markers and lines to create the lollipop effect
fig.add_trace(go.Scatter(
    x=df_top10["review_amount"],
    y=df_top10["name_and_subcategory"],
    mode="markers+lines",
    marker=dict(size=12),
    name="Reviews"
))

fig.update_layout(title="Top 10 Most Reviewed Products",
                  xaxis_title="Review Count", yaxis_title="Product")
fig = plot_preset(fig);


### Distribution of Size and Experience Related Feedback

In [20]:
# Aggregate total counts for all size-related feedback categories
df_size = df_main[[col for col in df_main.columns if "(size_ag)" in col]].sum().reset_index()
# Apply custom order to size dataframe
size_order = [
    "True to size (size_ag)",
    "Too Big (size_ag)",
    "Big (size_ag)",
    "Too Small (size_ag)",
    "Small (size_ag)",
    "Not specified (size_ag)"
]
df_size["index"] = pd.Categorical(df_size["index"], categories=size_order, ordered=True)
df_size = df_size.sort_values("index")
# Aggregate total counts for all experience-related feedback categories
df_exp = df_main[[col for col in df_main.columns if "(experience)" in col]].sum().reset_index()

# Plot distribution of size-related feedback on a logarithmic scale to handle wide value range
fig1 = px.bar(df_size, x = "index", y = 0, log_y=True, color = 0, color_continuous_scale= "Earth", title = "Distribution of Size-Related Feedback<br>(Log scale)")
fig1 = plot_preset(fig1)
fig1.update_layout(
    xaxis_title="Size Feedback Category",
    yaxis_title="Number of Responses",
    yaxis_tickvals = ["1000", "1500", "2500", "3500", "5000", "7000", "10000"],
    yaxis_ticktext = ["1000", "1500", "2500", "3500", "5000", "7000", "10000"],
    width=800, 
    height=500,    
)
fig1.update_coloraxes(colorbar_title_text='')

# Plot distribution of experience-related feedback on a logarithmic scale
fig2 = px.bar(df_exp, x = "index", y = 0, log_y=True, color = 0, color_continuous_scale= "Earth", title = "Distribution of Experience-Related Feedback<br>(Log scale)")
fig2 = plot_preset(fig2)
fig2.update_layout(
    xaxis_title="Experience Feedback Category",
    yaxis_title="Number of Responses",
    yaxis_tickvals = ["1000", "2000", "3500", "5000", "8000", "12000" ,"18000"],
    yaxis_ticktext = ["1000", "2000", "3500", "5000", "8000", "12000", "18000"],
    width=800, 
    height=500,  
)
fig2.update_coloraxes(colorbar_title_text='');

### Count of Verified Buyers, Reviewers and Unverified Buyers

In [21]:
df_main[['Verified Buyer',
       'Verified Reviewer', 'Unverified',]].sum().reset_index()

,index,0
0,Verified Buyer,18379.0
1,Verified Reviewer,569.0
2,Unverified,1243.0


### Normalized Size Feedback by Product Category

In [22]:
# Select size-related feedback columns and group by main [roduct category
df = df_main[["main_category"] + [col for col in df_main.columns if "(size_ag)" in col ]]

# Aggregate feedback counts by main category, summing responses per size feedback type
df = df.groupby("main_category").agg({
    "True to size (size_ag)": "sum",
    "Too Big (size_ag)" : "sum",
    "Small (size_ag)" : "sum",
    "Too Small (size_ag)" : "sum",
    "Not specified (size_ag)" : "sum",
    "Big (size_ag)" : "sum"
}).reset_index()

# Normalize feedback counts to percentages within each main category for relative comparison
df_wc = df.drop(["main_category"], axis =1)
df_sum = df_wc.sum(axis=1)
df_wc = df_wc.div(df_sum, axis=0) * 100
df_wc["main_category"] = df["main_category"]
df = df_wc

# Convert data to long format for easier plotting of grouped bar chart
df = df.melt(id_vars = "main_category", value_vars= ["True to size (size_ag)", "Too Big (size_ag)",	"Small (size_ag)", "Too Small (size_ag)", "Not specified (size_ag)", "Big (size_ag)"],
             var_name = "Size", value_name= "Count")

# Define order of size feedback categories and sort accordingly for consistent plot appearance
order = ["True to size (size_ag)", "Too Big (size_ag)", "Big (size_ag)", "Too Small (size_ag)", "Small (size_ag)", "Not specified (size_ag)"]
df["Size"] = pd.Categorical(df["Size"], categories=order, ordered=True)
df = df.sort_values(by=["main_category", "Size"])

# Create grouped bar chart showing normalized size feedback percentages by product category
fig = px.bar(df, x = "main_category", y = "Count", color = "Size", barmode="group", color_discrete_sequence=earth_palette, title = "Normalized Size Feedback by Product Category")
# Apply formatting
fig = plot_preset(fig)
fig.update_layout(
    width = 1500,
    xaxis_title = "Product Category",
    yaxis_title = "Feedback Share (%)");


### Average Rating by Product Category (Log Scale for Review Count)

In [23]:
# Aggregate total review counts and average ratings by main product category
df = df_main[["main_category", "master_name", "review_amount", "rating"]]
df = df.groupby("main_category").agg({
    "review_amount": "sum",     # Sum total reviews per category
    "rating": "mean"            # Calculate average rating per category
}).reset_index()

# Sort categories by average rating descending for better visual ranking
df = df.sort_values(by = "rating", ascending=False)

# Create horizontal bar chart: total reviewss with average rating labeled
fig = px.bar(df, x="review_amount", y="main_category", color="main_category", orientation="h", text="rating",  title="Average Rating by Category (Log Scale for Review Amount)",
    color_discrete_sequence=earth_palette, hover_data=["review_amount"]
)

# Configure log scale on x-axis with custom ticks and labels for readability
fig.update_layout(
    xaxis_title = "Number of Reviews (log scale)",
    yaxis_title = "Product Category",
    xaxis_type="log",
    xaxis_tickvals=[10, 100, 1000, 10000, 40000],
    xaxis_ticktext=["10", "100", "1K", "10K", "40K"],
    uniformtext_minsize=8,
    uniformtext_mode="hide",
    showlegend = False
)

# Display average rating on bars with two decimal precision outside bars
fig.update_traces(
    texttemplate="%{text:.2f}",  
    textposition="outside"       
);

### Unique Products vs Total Variants by Category (Grouped Bar Chart)

In [24]:
# Select relevant columns and group by main category
# Calculate number of unique master products and total product variants per category
df = df_variants[["main_category", "master_name", "full_product_name"]]
# Rename columns for clarity
df = df.rename(columns={
    "master_name" : "Unique products",
    "full_product_name" : "Total variants"
}
)
df = df.groupby("main_category").agg({
    "Unique products" : pd.Series.nunique, # count unique master products
    "Total variants" : "count" # count total variants
}).reset_index()

# Transform data to long format for grouped bar plotting
df = df.melt(id_vars="main_category", value_vars=["Unique products", "Total variants"], var_name="type", value_name="count")

# Create grouped bar chart comparing unique products and variants by category (log scale)
fig = px.bar(df, x = "main_category", y ="count", log_y=True, color = "type",  color_discrete_sequence=earth_palette, barmode= "group",
              title= "Unique Products vs Total Variants by Category(Log scale for Product Amount)")

fig = plot_preset(fig)
fig.update_layout(
    xaxis_title = "Category name",
    yaxis_title = "Product Amount",
    yaxis_tickvals = ["50", "75", "100", "150", "225", "350", "500", "700", "1000", "1500", "2200"]
);

### Unique Products vs Total Variants by Category (Dumbbell Chart)

In [25]:
# Group and aggregate data to get the number of unique master products and total variants per category
df_dumbbell = df_variants.groupby("main_category").agg({
    "master_name": pd.Series.nunique,         # Count of distinct master products
    "full_product_name": "count"              # Total number of product variants
}).reset_index().rename(columns={
    "master_name": "Unique Products",
    "full_product_name": "Total Variants"
})

# Sort categories by total variant count for cleaner visualization
df_dumbbell = df_dumbbell.sort_values("Total Variants")

# Initialize Plotly figure
fig = go.Figure()

# Add connecting lines between unique product and total variant counts
for i, row in df_dumbbell.iterrows():
    fig.add_trace(go.Scatter(
        x=[row["Unique Products"], row["Total Variants"]],        
        y=[row["main_category"], row["main_category"]],
        mode='lines',
        line=dict(color='gray', width=3),
        showlegend=False
    ))

# Add left-side markers for unique product counts
fig.add_trace(go.Scatter(
    x=df_dumbbell["Unique Products"],
    y=df_dumbbell["main_category"],
    mode='markers',
    marker=dict(color='blue', size=10),
    name='Unique Products'
))

# Add right-side markers for total variant counts
fig.add_trace(go.Scatter(
    x=df_dumbbell["Total Variants"],
    y=df_dumbbell["main_category"],
    mode='markers',
    marker=dict(color='orange', size=10),
    name='Total Variants'
))

fig.update_layout(
    title="Dumbbell Chart: Unique Products vs Total Variants by Category",
    xaxis_title="Count",
    yaxis_title="Category",
    height=600
);

### Average Price by Product Category

In [26]:
# Calculate mean price for each main product category
df = df_variants[["main_category", "price"]]
df = df.groupby("main_category")["price"].mean().reset_index()
# Sort categories by descending average price for clearer visualization
df = df.sort_values("price", ascending=False).reset_index(drop=True)

# Create horizontal bar chart of average prices by category
fig = px.bar(df, x = "price", y ="main_category", color="main_category", color_discrete_sequence=earth_palette, title = "Average Price by Product Category")

fig = plot_preset(fig)
fig.update_layout(
    xaxis_title = "Average Price (USD)",
    yaxis_title = "Product Category",
    yaxis={'categoryorder':'total descending'},
    width = 1100
)
# Manually add price labels outside bars for clarity
price_labels = ["116.65", "91.67", "76.5", "73.09", "48.02", "23.25"]
for i, trace in enumerate(fig.data):
    trace.text = price_labels[i]
    trace.textposition = "outside"

### Price Distribution by Category (Violin Plot)

In [27]:
# Create violin plot showing price distribution across product categories
fig = px.violin(df_variants, y="main_category", x="price", color="main_category", box=True,
    points="outliers",  # Display only outlier points for clarity
    title="Violin Plot: Price Distribution by Category",
    hover_data={"price": ":.2f", "main_category": True},  # Format price hover info
    color_discrete_sequence=px.colors.qualitative.Set2  # Set consistent color scheme
)

# Refine marker appearance and point distribution
fig.update_traces(
    jitter=0.2,  # Moderate horizontal spread of points
    pointpos=0.0,  # Center points within violins
    marker=dict(size=3, opacity=0.7)  # Smaller, semi-transparent points
)

# Customize layout for clarity and visual appeal
fig.update_layout(
    xaxis_title="Price ($)",
    yaxis_title="Product Category",
    title_x=0.5,  # Center plot title
    font=dict(family="Arial", size=12, color="white")
);

### Product Count Distribution Across Price Ranges by Category (Log Scale)

In [28]:
# Categorize product prices into defined ranges for segmentation analysis
df = df_variants[["main_category", "price"]].copy()
bins = [0, 15, 30, 50, 75, 100, float("inf")]
labels = ["<15", "15-30", "30-50", "50-75", "75-100", ">100"]
df["price_range"] = pd.cut(df["price"], bins=bins, labels=labels, right=False)

# Calculate count of products in each price range within each main category
df = df.groupby(["main_category", "price_range"], observed=True).size().reset_index(name="count")
# Visualize price distribution across categories using a grouped bar chart with log scale
fig = px.bar(df, x = "main_category", y = "count", log_y=True, color = "price_range", color_discrete_sequence=earth_palette, barmode="group", title="Product Count Distribution Across Price Bins by Category (Log Scale)")

fig = plot_preset(fig)
fig.update_layout(
    xaxis_title = "Product Category",
    yaxis_title = "Product Amount",
    yaxis_tickvals = ["1", "3", "10", "25", "50", "100", "200", "500", "1000"]
);